In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 13.1 MB/s eta 0:00:00


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2023-11-08 04:34:06--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-11-08 04:34:06--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-11-08 04:34:07--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
!wget https://raw.githubusercontent.com/sighsmile/conlleval/master/conlleval.py

--2023-11-08 04:37:09--  https://raw.githubusercontent.com/sighsmile/conlleval/master/conlleval.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7502 (7.3K) [text/plain]
Saving to: ‘conlleval.py’

conlleval.py        100%[===================>]   7.33K  --.-KB/s    in 0s      

2023-11-08 04:37:09 (104 MB/s) - ‘conlleval.py’ saved [7502/7502]



In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam,SGD
from torch.nn.functional import cross_entropy
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import numpy as np
import pandas as pd

In [ ]:
import datasets

dataset = datasets.load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
def load_embeddings(filename):
    word2vec = {}
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip().split()
            word = line[0]
            embedding = [float(x) for x in line[1:]]
            word2vec[word] = np.array(embedding)
    return word2vec

word2vec = load_embeddings('glove.6B.100d.txt')
keys = list(word2vec.keys())
for i in keys:
    word2vec[i] = np.append(word2vec[i],-1.0)#non-capitalized
    word2vec[i.title()] = np.append(word2vec[i][:-1],0.0) # title
    word2vec[i.upper()] = np.append(word2vec[i][:-1],1.0) # fully capitalized

In [ ]:
word2vec['happy']

array([-0.090436 ,  0.19636  ,  0.29474  , -0.47706  , -0.80436  ,
        0.3078   , -0.55205  ,  0.58453  , -0.17056  , -0.84846  ,
        0.19528  ,  0.23671  ,  0.46827  , -0.58977  , -0.12163  ,
       -0.24697  , -0.072944 ,  0.17259  , -0.0485   ,  0.9527   ,
        0.50629  ,  0.58497  , -0.19367  , -0.45459  , -0.031095 ,
        0.51633  , -0.24052  , -0.1007   ,  0.53627  ,  0.024225 ,
       -0.50162  ,  0.73692  ,  0.49468  , -0.34744  ,  0.89337  ,
        0.057439 , -0.19127  ,  0.39333  ,  0.21182  , -0.89837  ,
        0.078704 , -0.16344  ,  0.45261  , -0.41096  , -0.19499  ,
       -0.13489  , -0.016313 , -0.021849 ,  0.17136  , -1.2413   ,
        0.079503 , -0.91144  ,  0.35699  ,  0.36289  , -0.24934  ,
       -2.1196   ,  0.14534  ,  0.52964  ,  0.90134  ,  0.033603 ,
        0.022809 ,  0.70625  , -1.0362   , -0.59809  ,  0.70592  ,
       -0.072793 ,  0.67033  ,  0.52763  , -0.47807  , -0.67374  ,
        0.36632  , -0.38284  , -0.10349  , -0.6402   ,  0.1810

In [ ]:
word2vec['Happy']

array([-0.090436 ,  0.19636  ,  0.29474  , -0.47706  , -0.80436  ,
        0.3078   , -0.55205  ,  0.58453  , -0.17056  , -0.84846  ,
        0.19528  ,  0.23671  ,  0.46827  , -0.58977  , -0.12163  ,
       -0.24697  , -0.072944 ,  0.17259  , -0.0485   ,  0.9527   ,
        0.50629  ,  0.58497  , -0.19367  , -0.45459  , -0.031095 ,
        0.51633  , -0.24052  , -0.1007   ,  0.53627  ,  0.024225 ,
       -0.50162  ,  0.73692  ,  0.49468  , -0.34744  ,  0.89337  ,
        0.057439 , -0.19127  ,  0.39333  ,  0.21182  , -0.89837  ,
        0.078704 , -0.16344  ,  0.45261  , -0.41096  , -0.19499  ,
       -0.13489  , -0.016313 , -0.021849 ,  0.17136  , -1.2413   ,
        0.079503 , -0.91144  ,  0.35699  ,  0.36289  , -0.24934  ,
       -2.1196   ,  0.14534  ,  0.52964  ,  0.90134  ,  0.033603 ,
        0.022809 ,  0.70625  , -1.0362   , -0.59809  ,  0.70592  ,
       -0.072793 ,  0.67033  ,  0.52763  , -0.47807  , -0.67374  ,
        0.36632  , -0.38284  , -0.10349  , -0.6402   ,  0.1810

In [ ]:
word2vec['HAPPY']

array([-0.090436 ,  0.19636  ,  0.29474  , -0.47706  , -0.80436  ,
        0.3078   , -0.55205  ,  0.58453  , -0.17056  , -0.84846  ,
        0.19528  ,  0.23671  ,  0.46827  , -0.58977  , -0.12163  ,
       -0.24697  , -0.072944 ,  0.17259  , -0.0485   ,  0.9527   ,
        0.50629  ,  0.58497  , -0.19367  , -0.45459  , -0.031095 ,
        0.51633  , -0.24052  , -0.1007   ,  0.53627  ,  0.024225 ,
       -0.50162  ,  0.73692  ,  0.49468  , -0.34744  ,  0.89337  ,
        0.057439 , -0.19127  ,  0.39333  ,  0.21182  , -0.89837  ,
        0.078704 , -0.16344  ,  0.45261  , -0.41096  , -0.19499  ,
       -0.13489  , -0.016313 , -0.021849 ,  0.17136  , -1.2413   ,
        0.079503 , -0.91144  ,  0.35699  ,  0.36289  , -0.24934  ,
       -2.1196   ,  0.14534  ,  0.52964  ,  0.90134  ,  0.033603 ,
        0.022809 ,  0.70625  , -1.0362   , -0.59809  ,  0.70592  ,
       -0.072793 ,  0.67033  ,  0.52763  , -0.47807  , -0.67374  ,
        0.36632  , -0.38284  , -0.10349  , -0.6402   ,  0.1810

In [ ]:
import itertools
from collections import Counter
# REFERENCE CODE PROVIDED BY SHOUMIK
# word_frequency = Counter(itertools.chain(*dataset['train']['tokens']))  # type: ignore

# # Remove words below threshold 2
# word2idx = {
#     word: frequency
#     for word, frequency in word_frequency.items()
#     if frequency >= 2
# }

# word2idx = {
#     word: index
#     for index, word in enumerate(word_frequency.keys(), start=2)
# }
word2idx = {word: i+4 for i, word in enumerate(word2vec)} # + 4 for padding, unknown lower, unknown title, unknown upper
word2idx['[PAD]'] = 0 #padding
word2idx['[UNKL]'] = 1 #unkown lowercase word
word2idx['[UNKT]'] = 2 #unkown titlecase word
word2idx['[UNKU]'] = 3 #unkown uppercase word

In [ ]:
def which_unknown(word2idx,word):
    """
    decides which unkown case to assign a particular word
    """
    if word.istitle():
        return word2idx['[UNKT]']
    if word.isupper():
        return word2idx['[UNKU]']
    #if nothing else, lets just assume its a lower case:
    return word2idx['[UNKL]']

dataset = (
    dataset
    .map(lambda x: {
            'input_ids': [
                word2idx.get(word,which_unknown(word2idx,word)) #word2idx['[UNK]']
                for word in x['tokens']
            ]
        }
    )
)

dataset['train']['input_ids'][:3]

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

[[401213, 7582, 400966, 584, 8, 5264, 400551, 10242, 6],
 [402419, 417195],
 [407382, 1]]

In [ ]:
dataset['train']['tokens'][:3]

[['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
 ['Peter', 'Blackburn'],
 ['BRUSSELS', '1996-08-22']]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids'],
        num_rows: 3453
    })
})

In [ ]:
columns_to_remove = ['pos_tags', 'chunk_tags']
for split in dataset.keys():
    dataset[split] = dataset[split].remove_columns(columns_to_remove)

# Rename ner_tags to labels
for split in dataset.keys():
    dataset[split] = dataset[split].rename_column('ner_tags', 'labels')

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'labels', 'input_ids'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'labels', 'input_ids'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'labels', 'input_ids'],
        num_rows: 3453
    })
})


In [ ]:
label2id = dataset["train"].features["labels"].feature
id2label = {id: label for label, id in enumerate(label2id.names)}
id2label['PAD'] = -1
id2label

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8,
 'PAD': -1}

In [ ]:
# creating embedding matrix for model
embedding_matrix = np.zeros((len(word2idx), 101))
random_embeddings = np.random.randn(101)
avg_vector = np.mean(np.array(list(word2vec.values())), axis=0) # Average vector for unknown words

# Fill in embedding matrix with GloVe vectors
embedding_matrix[word2idx['[PAD]']] = np.zeros(101)#np.random.randn(101)
# embedding_matrix[word2idx['[UNK]']] = random_embeddings
embedding_matrix[word2idx['[UNKL]']] = np.append(avg_vector[:-1],-1.0)
embedding_matrix[word2idx['[UNKT]']] = np.append(avg_vector[:-1],0.0)
embedding_matrix[word2idx['[UNKU]']] = np.append(avg_vector[:-1],1.0)
for i, word in enumerate(word2vec):
    embedding_matrix[i+4] = word2vec[word]

# converting into torch tensor and Freezeing the embeddings:
embedding_matrix = torch.from_numpy(embedding_matrix.astype('float32'))
embedding_matrix.requires_grad = False

 Task 1: Bidirectional LSTM model

In [ ]:
class BiLSTMNER(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_dim=101, hidden_dim=256, linear_dim=128, dropout=0.33, num_layers=1):
        super(BiLSTMNER, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=True)
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers=num_layers,
                            dropout=dropout if num_layers > 1 else 0,
                            batch_first = True,
                            bidirectional=True)
        self.linear = nn.Linear(2*hidden_dim, linear_dim)
        self.elu = nn.ELU(alpha = 0.5) #0.75
        self.dropout = nn.Dropout(p=dropout)
        self.classifier = nn.Linear(linear_dim, tagset_size)

    def forward(self, sentence, lengths):
        embedded = self.embedding(sentence)

        # Pack the embeddings
        packed_embedded = pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)

        packed_lstm_out, _ = self.lstm(packed_embedded)

        # Unpack the sequence
        lstm_out, _ = pad_packed_sequence(packed_lstm_out, batch_first=True)
        lstm_out = self.dropout(lstm_out)
        linear_out = self.elu(self.linear(lstm_out))
        tag_space = self.classifier(linear_out)

        return tag_space.permute(0,2,1)


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence

def preprocess_data(data):
    input_ids = [torch.tensor(seq) for seq in data['input_ids']]
    labels = [torch.tensor(label) for label in data['labels']]
    return list(zip(input_ids, labels))

def dynamic_padding(batch):
    inputs = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    lengths = torch.tensor([len(inp) for inp in inputs])

    # Dynamic padding in the batch
    inputs = pad_sequence(inputs, batch_first=True)
    labels = pad_sequence(labels, batch_first=True, padding_value=-1)

    return inputs, labels, lengths


# Hyperparameters
BATCH_SIZE = 16#32
# Preprocess the train, val, and test data
train_data = preprocess_data(dataset['train'])
val_data = preprocess_data(dataset['validation'])
test_data = preprocess_data(dataset['test'])

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=dynamic_padding, num_workers=2)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, collate_fn=dynamic_padding, num_workers=2)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, collate_fn=dynamic_padding, num_workers=2)

In [ ]:
from collections import Counter

vocab_size = max([max(seq) for seq in dataset['train']['input_ids']]) + 1
tagset_size = max([max(seq) for seq in dataset['train']['labels']]) + 1
# Device definition
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
all_labels = [label for sublist in dataset['train']['labels'] for label in sublist]
label_counts = {label: all_labels.count(label) for label in id2label.values() if label != -1}
total_labels = len(all_labels) # We subtract the count of 'PAD' labels
weights = [total_labels / label_counts[id2label[key]] for key in id2label.keys() if key != 'PAD']
weights = [weight / sum(weights) for weight in weights]

weights_tensor = torch.tensor(weights).to(device)

In [ ]:
label_counts

{0: 169578,
 1: 6600,
 2: 4528,
 3: 6321,
 4: 3704,
 5: 7140,
 6: 1157,
 7: 3438,
 8: 1155}

In [ ]:
weights

[0.0019872122378459785,
 0.05105870861658262,
 0.07442302934395877,
 0.05331236780089311,
 0.09097934040751764,
 0.04719712561196713,
 0.2912597034308084,
 0.09801846331281132,
 0.291764049237615]

In [ ]:
sample_batch = next(iter(train_loader))
len(sample_batch)

3

In [ ]:
model = BiLSTMNER(vocab_size, tagset_size)
model

BiLSTMNER(
  (embedding): Embedding(1127234, 101)
  (lstm): LSTM(101, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=128, bias=True)
  (elu): ELU(alpha=0.5)
  (dropout): Dropout(p=0.33, inplace=False)
  (classifier): Linear(in_features=128, out_features=9, bias=True)
)

In [ ]:
LEARNING_RATE = 75e-2#75e-3#1e-1 #75e-2 #3e-1 #75e-2#1.0 #5e-1#1e-1
loss_function = torch.nn.CrossEntropyLoss(weight=weights_tensor, ignore_index=-1).to(device)
# Optimizer
# optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
optimizer = SGD(model.parameters(), lr=LEARNING_RATE)
# Move model to the device
model = model.to(device)

In [ ]:
# Loss function
# loss_function = torch.nn.CrossEntropyLoss(ignore_index=-1).to(device)
#300 so far
EPOCHS = 50#100
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    # Wrap your training loader with tqdm for progress bar
    for inputs, targets, lengths in train_loader: #tqdm(train_loader, desc=f"Epoch {epoch + 1}/{EPOCHS}"):
        optimizer.zero_grad()

        # Fetch inputs and targets and move them to the current device
        inputs = inputs.to(device)
        targets = targets.to(device)
        lengths = lengths.to(device)

        # Forward pass
        outputs = model(inputs, lengths)

        # Compute loss and backpropagate
        loss = loss_function(outputs,targets) #(outputs.view(-1, tagset_size), targets.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets, lengths in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            lengths = lengths.to(device)

            outputs = model(inputs, lengths)

            loss = loss_function(outputs,targets) #(outputs.view(-1, tagset_size), targets.view(-1))
            val_loss += loss.item()

    print(f"Epoch {epoch + 1}/{EPOCHS}, Training Loss: {total_loss / len(train_loader)} Validation Loss: {val_loss / len(val_loader)}")


Epoch 1/50, Training Loss: 0.8141673816143109 Validation Loss: 0.46101467019202663
Epoch 2/50, Training Loss: 0.4019707834861477 Validation Loss: 0.38149524649020794
Epoch 3/50, Training Loss: 0.29123258200348917 Validation Loss: 0.3048748771897426
Epoch 4/50, Training Loss: 0.23722692021298178 Validation Loss: 0.2586067534033574
Epoch 5/50, Training Loss: 0.18413354061726386 Validation Loss: 0.2921877385244942
Epoch 6/50, Training Loss: 0.1530364083634278 Validation Loss: 0.2612298799258675
Epoch 7/50, Training Loss: 0.12721821191139385 Validation Loss: 0.267946543714988
Epoch 8/50, Training Loss: 0.10603927182943766 Validation Loss: 0.2779435542590149
Epoch 9/50, Training Loss: 0.09258149781106316 Validation Loss: 0.3695681261089311
Epoch 10/50, Training Loss: 0.07892728415593624 Validation Loss: 0.27078670984152337
Epoch 11/50, Training Loss: 0.06462572047042454 Validation Loss: 0.3118343375122005
Epoch 12/50, Training Loss: 0.05058781890124867 Validation Loss: 0.33446648336122925
E

In [ ]:
def get_predictions(model, loader, device):
    model.eval()
    all_predictions = []
    with torch.no_grad():
        for inputs, _, lengths in loader: # We don't need targets now
            inputs = inputs.to(device)

            outputs = model(inputs, lengths)
            # Get predictions
            predictions = torch.argmax(outputs, dim=1)

            # Truncate predictions to their original lengths
            truncated_predictions = [pred[:len_].tolist() for pred, len_ in zip(predictions, lengths)]

            all_predictions.extend(truncated_predictions)

    return all_predictions

In [ ]:
val_predictions = get_predictions(model, val_loader, device)
test_predictions = get_predictions(model, test_loader, device)

In [ ]:
from conlleval import evaluate
import itertools
# labels = ner_tags
# Map the labels back to their corresponding tag strings
idx2tag  = {id:tag for (tag,id) in id2label.items()}
labels = [
list(map(idx2tag.get, labels))
for labels in dataset['validation']['labels']
]
# This is the prediction by your model
preds = [
list(map(idx2tag.get, labels))
for labels in val_predictions
]
precision, recall, f1 = evaluate(itertools.chain(*labels),itertools.chain(*preds))

processed 51362 tokens with 5942 phrases; found: 6170 phrases; correct: 5514.
accuracy:  93.78%; (non-O)
accuracy:  98.38%; precision:  89.37%; recall:  92.80%; FB1:  91.05
              LOC: precision:  93.48%; recall:  96.84%; FB1:  95.13  1903
             MISC: precision:  78.54%; recall:  86.12%; FB1:  82.15  1011
              ORG: precision:  85.95%; recall:  87.62%; FB1:  86.78  1367
              PER: precision:  93.49%; recall:  95.87%; FB1:  94.67  1889


In [ ]:
from conlleval import evaluate
import itertools
# labels = ner_tags
# Map the labels back to their corresponding tag strings
idx2tag  = {id:tag for (tag,id) in id2label.items()}
labels = [
list(map(idx2tag.get, labels))
for labels in dataset['test']['labels']
]
# This is the prediction by your model
preds = [
list(map(idx2tag.get, labels))
for labels in test_predictions
]
precision, recall, f1 = evaluate(itertools.chain(*labels),itertools.chain(*preds))

processed 46435 tokens with 5648 phrases; found: 6015 phrases; correct: 5010.
accuracy:  90.84%; (non-O)
accuracy:  97.26%; precision:  83.29%; recall:  88.70%; FB1:  85.91
              LOC: precision:  86.04%; recall:  93.82%; FB1:  89.76  1819
             MISC: precision:  63.13%; recall:  75.36%; FB1:  68.70  838
              ORG: precision:  81.09%; recall:  84.17%; FB1:  82.60  1724
              PER: precision:  92.90%; recall:  93.88%; FB1:  93.39  1634


In [ ]:
# code to save pytorch model
# Move the model to CPU
model.to('cpu')

# Save the model's state_dict
torch.save(model.state_dict(), 'model_hw4_task2.pth')